# FLAT input data prepping

This code prepares the final input file to the FLAT model. 

In [1]:
#Import modules and libraries

import geopandas as gpd
import rasterio as rio
import pandas as pd
import fiona
import gdal
import osr
import ogr
import rasterio.mask
import time
import os
import ogr, gdal, osr, os
import numpy as np
import itertools
import re

from osgeo import gdal,ogr
import struct
import csv

import glob

from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling
from rasterstats import point_query
from pyproj import Proj
from shapely.geometry import Point, Polygon

In [2]:
# Import data 

path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
name_of_file = "Moz_Maize_2000_admin2_10km.csv"

In [3]:
# Import csv as pandas dataframe
df = pd.read_csv(path + "\\" + name_of_file)

In [4]:
#df = df.drop(["yield", "production_ha"], axis=1)
df.rename(columns={'id': 'alloc_key'}, inplace=True)

In [5]:
df.rename(columns={'COUNTRY_COUNTRY': 'state'}, inplace=True)
df.rename(columns={'X_deg': 'lon'}, inplace=True)
df.rename(columns={'Y_deg': 'lat'}, inplace=True)
df.rename(columns={'area_ha': 'statearea_ha'}, inplace=True)
df.rename(columns={'country_code': 'c_code'}, inplace=True)

In [6]:
df.head()

,alloc_key,state,c_code,country,crop,year,harv_area_ha,yield,production_ha,statearea_ha,perimeter_km,lon,lat
0,10970,Guro,MOZ,Mozambique,Maize,2000,7318.0,0.024597,180.0,691032.666584,361.288935,33.584479,-16.454303
1,10971,Guro,MOZ,Mozambique,Maize,2000,7318.0,0.024597,180.0,691032.666584,361.288935,33.677746,-16.456699
2,10972,Guro,MOZ,Mozambique,Maize,2000,7318.0,0.024597,180.0,691032.666584,361.288935,33.771029,-16.459054
3,11109,Guro,MOZ,Mozambique,Maize,2000,7318.0,0.024597,180.0,691032.666584,361.288935,33.488673,-16.541846
4,11110,Guro,MOZ,Mozambique,Maize,2000,7318.0,0.024597,180.0,691032.666584,361.288935,33.581967,-16.544297


In [7]:
#Function to change the order of columns in the dataframe 
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

In [8]:
# Allocate order in dataframe columns
df = change_column_order(df, 'alloc_key', 1)
df = change_column_order(df, 'lon', 2)
df = change_column_order(df, 'lat', 3)
df = change_column_order(df, 'country', 4)
df = change_column_order(df, 'c_code', 5)
df = change_column_order(df, 'state', 6)
df = change_column_order(df, 'statearea_ha', 7)
df = change_column_order(df, 'perimeter_km', 8)
df = change_column_order(df, 'year', 9)
df = change_column_order(df, 'crop', 10)
df = change_column_order(df, 'harv_area_ha', 11)

In [9]:
df.head()

,alloc_key,lon,lat,country,c_code,state,statearea_ha,perimeter_km,year,crop,yield,harv_area_ha,production_ha
0,10970,33.584479,-16.454303,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0
1,10971,33.677746,-16.456699,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0
2,10972,33.771029,-16.459054,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0
3,11109,33.488673,-16.541846,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0
4,11110,33.581967,-16.544297,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0


In [10]:
# Add geometry and convert to spatial dataframe in source CRS
df['geometry'] = list(zip(df['lon'], df['lat']))
df['geometry'] = df['geometry'].apply(Point)
df = gpd.GeoDataFrame(df, geometry='geometry', crs={'init': 'epsg:4326'})

# Reproject data in to Ordnance Survey GB coordinates
df_osgb = df.to_crs({'init': 'epsg:32737'})

In [11]:
df_osgb.head()

,alloc_key,lon,lat,country,c_code,state,statearea_ha,perimeter_km,year,crop,yield,harv_area_ha,production_ha,geometry
0,10970,33.584479,-16.454303,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0,POINT (-78816.442 8173046.256)
1,10971,33.677746,-16.456699,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0,POINT (-68816.442 8173046.256)
2,10972,33.771029,-16.459054,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0,POINT (-58816.442 8173046.256)
3,11109,33.488673,-16.541846,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0,POINT (-88816.442 8163046.256)
4,11110,33.581967,-16.544297,Mozambique,MOZ,Guro,691032.666584,361.288935,2000,Maize,0.024597,7318.0,180.0,POINT (-78816.442 8163046.256)


<div class="alert alert-block alert-warning">
<b>Note:</b> The geodataframe, contains crop points with needed attributes including a geometry column. It needs to be exported as shapefile. However, when this is done with geopandas, the column geometry (Point(x,y)) is not transfered in the attribute table. This is however needed for the extraction process below, otherwise "None" values are returned. In order to overcome this issue, the csv is imported into Qgis and then exported as feature layer in the same folder. This solves the issue. 
</div>

In [12]:
# Define output path
# Overwriting the csv file
path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
shpname = "Moz_Maize_2000_admin2_10km_geom.csv"

#drybeans
df_osgb.to_csv(os.path.join(path,"{c}".format(c=shpname)), index=False)

#### >>> Please open Qgis and act as suggested above<<<

## Extracting raster input to csv 

In [14]:
raster_path = r"N:\Agrodem\Downscaling\Input_Data\rasters"
raster_files =[]
csvoutpath = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input\10km_Maize"

print ("Reading independent variables...")

for i in os.listdir(raster_path):
    if i.endswith('.tif'):
        raster_files.append(i)  

shp_filename = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops\shapefiles\Moz_Maize_2000_admin2_10km_geom.shp"

print ("Extracting raster values to points...")

for i in raster_files:
    print("Extracting " + i + " values...")
    src_filename = raster_path + "\\" + i 
    li_values = list()

    src_ds=gdal.Open(src_filename) 
    gt=src_ds.GetGeoTransform()
    rb=src_ds.GetRasterBand(1)
    ds=ogr.Open(shp_filename)
    lyr=ds.GetLayer()
    
    for feat in lyr:
        geom = feat.GetGeometryRef()
        feat_id = feat.GetField('alloc_key')
        mx,my=geom.GetX(), geom.GetY()  #coord in map units

        #Convert from map to pixel coordinates.
        #Only works for geotransforms with no rotation.
        px = int((mx - gt[0]) / gt[1]) #x pixel
        py = int((my - gt[3]) / gt[5]) #y pixel

        intval=rb.ReadAsArray(px,py,1,1)
        li_values.append([feat_id, intval[0]])
        
    print ("Writing " + i + " values to csv...")
    
    with open(csvoutpath + "\\" + i.split('.')[0] + '.csv', 'w') as csvfile:
        wr = csv.writer(csvfile)
        wr.writerows(li_values) 

Reading independent variables...
Extracting raster values to points...
Extracting BDRICM.tif values...
Writing BDRICM.tif values to csv...
Extracting BLD.tif values...
Writing BLD.tif values to csv...
Extracting CLYPPT.tif values...
Writing CLYPPT.tif values to csv...
Extracting DRAINFAO.tif values...
Writing DRAINFAO.tif values to csv...
Extracting elevation.tif values...
Writing elevation.tif values to csv...
Extracting EVI.tif values...
Writing EVI.tif values to csv...
Extracting Landcover30m.tif values...
Writing Landcover30m.tif values to csv...
Extracting NDVI.tif values...
Writing NDVI.tif values to csv...
Extracting ORCDRC.tif values...
Writing ORCDRC.tif values to csv...
Extracting PHIHOX.tif values...
Writing PHIHOX.tif values to csv...
Extracting prec.tif values...
Writing prec.tif values to csv...
Extracting slope.tif values...
Writing slope.tif values to csv...
Extracting srad.tif values...
Writing srad.tif values to csv...
Extracting tavg.tif values...
Writing tavg.tif va

## Merge csv files with crop

In [15]:
#Import data 

path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
name_of_file = "Moz_Maize_2000_admin2_10km_geom.csv"

flat_input = pd.read_csv(path + "\\" + name_of_file)

In [16]:
csv_files = []

print ("Reading csv files...")

for i in os.listdir(csvoutpath):
    if i.endswith('.csv'):
        csv_files.append(i)  

for i in csv_files:
    print('Reading...'+ i)    
    df_csv = pd.read_csv(csvoutpath + "//" + i, index_col=None, header=None)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].astype(str)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace('[','')
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace(']','')
    columnName = i.split('.')[0]
    print("Merging..." + columnName)
    flat_input[columnName] = df_csv.iloc[:,1]

Reading csv files...
Reading...BDRICM.csv
Merging...BDRICM
Reading...BLD.csv
Merging...BLD
Reading...CLYPPT.csv
Merging...CLYPPT
Reading...DRAINFAO.csv
Merging...DRAINFAO
Reading...elevation.csv
Merging...elevation
Reading...EVI.csv
Merging...EVI
Reading...Landcover30m.csv
Merging...Landcover30m
Reading...NDVI.csv
Merging...NDVI
Reading...ORCDRC.csv
Merging...ORCDRC
Reading...PHIHOX.csv
Merging...PHIHOX
Reading...prec.csv
Merging...prec
Reading...slope.csv
Merging...slope
Reading...srad.csv
Merging...srad
Reading...tavg.csv
Merging...tavg
Reading...TEXMHT.csv
Merging...TEXMHT
Reading...wind.csv
Merging...wind


## Cleaning and normalizing dataframe

In [17]:
#Show columns
flat_input.columns

Index(['alloc_key', 'lon', 'lat', 'country', 'c_code', 'state', 'statearea_ha',
       'perimeter_km', 'year', 'crop', 'yield', 'harv_area_ha',
       'production_ha', 'geometry', 'BDRICM', 'BLD', 'CLYPPT', 'DRAINFAO',
       'elevation', 'EVI', 'Landcover30m', 'NDVI', 'ORCDRC', 'PHIHOX', 'prec',
       'slope', 'srad', 'tavg', 'TEXMHT', 'wind'],
      dtype='object')

In [18]:
#Delete geometry column
flat_input = flat_input.drop(["geometry"], axis=1)

#### Convert values in newly added columns into float 

In [19]:
flat_input["harv_area_ha"] = flat_input["harv_area_ha"].astype(float)
flat_input["BDRICM"] = flat_input["BDRICM"].astype(float)
flat_input["BLD"] = flat_input["BLD"].astype(float)
flat_input["CLYPPT"] = flat_input["CLYPPT"].astype(float)
flat_input["DRAINFAO"] = flat_input["DRAINFAO"].astype(float)
flat_input["elevation"] = flat_input["elevation"].astype(float)
flat_input["EVI"] = flat_input["EVI"].astype(float)
flat_input["Landcover30m"] = flat_input["Landcover30m"].astype(float)
flat_input["NDVI"] = flat_input["NDVI"].astype(float)
flat_input["ORCDRC"] = flat_input["ORCDRC"].astype(float)
flat_input["PHIHOX"] = flat_input["PHIHOX"].astype(float)
flat_input["prec"] = flat_input["prec"].astype(float)
flat_input["slope"] = flat_input["slope"].astype(float)
flat_input["srad"] = flat_input["srad"].astype(float)
flat_input["tavg"] = flat_input["tavg"].astype(float)
flat_input["TEXMHT"] = flat_input["TEXMHT"].astype(float)
flat_input["wind"] = flat_input["wind"].astype(float)

In [20]:
flat_input.dtypes

alloc_key          int64
lon              float64
lat              float64
country           object
c_code            object
state             object
statearea_ha     float64
perimeter_km     float64
year               int64
crop              object
yield            float64
harv_area_ha     float64
production_ha    float64
BDRICM           float64
BLD              float64
CLYPPT           float64
DRAINFAO         float64
elevation        float64
EVI              float64
Landcover30m     float64
NDVI             float64
ORCDRC           float64
PHIHOX           float64
prec             float64
slope            float64
srad             float64
tavg             float64
TEXMHT           float64
wind             float64
dtype: object

#### Normalize EVI and NDVI 

In [21]:
flat_input['EVI'] = flat_input['EVI']*0.0001
flat_input['NDVI'] = flat_input['NDVI']*0.0001

#### Keep areas that are indicated as cropland in GFSAD30m

In [22]:
flat_input = flat_input[flat_input["Landcover30m"] == 2]
flat_input = flat_input.drop(["Landcover30m"], axis=1)

#### Fixing out-of-range values

In [23]:
flat_input['tavg'][flat_input['tavg']<0] = flat_input['tavg'].mean()
flat_input['prec'][flat_input['prec']<0] = flat_input['prec'].mean()
flat_input['wind'][flat_input['wind']<0] = flat_input['wind'].mean()
flat_input['BLD'][flat_input['BLD']<0] = 0
flat_input['ORCDRC'][flat_input['ORCDRC']<0] = 0

flat_input.fillna(0,inplace=True)

C:\Users\adm.esa\Anaconda3\envs\testgdal\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\adm.esa\Anaconda3\envs\testgdal\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\adm.esa\Anaconda3\envs\testgdal\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Thi

In [24]:
flat_input

,alloc_key,lon,lat,country,c_code,state,statearea_ha,perimeter_km,year,crop,...,EVI,NDVI,ORCDRC,PHIHOX,prec,slope,srad,tavg,TEXMHT,wind
76,8082,38.453855,-14.625298,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,Maize,...,0.3034,0.5880,11.0,62.0,87.583336,0.785641,17325.250,24.358332,9.0,1.2
91,8497,37.988363,-14.894948,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,Maize,...,0.3229,0.5771,12.0,61.0,90.083336,0.821979,16929.334,23.341667,9.0,1.2
98,8636,37.894945,-14.984914,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,Maize,...,0.3155,0.5116,10.0,62.0,92.083336,1.964763,16879.000,22.891668,9.0,1.3
100,8638,38.080936,-14.985740,Mozambique,MOZ,Ribaue,644555.756771,406.138864,2000,Maize,...,0.3180,0.4625,9.0,62.0,93.750000,1.473183,16906.166,23.041666,9.0,1.2
149,9772,39.384372,-15.710531,Mozambique,MOZ,Mogovolas,454898.198313,335.429510,2000,Maize,...,0.3383,0.5690,8.0,64.0,88.416664,1.323050,18582.834,25.283333,9.0,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5373,11858,38.071366,-17.064756,Mozambique,MOZ,Pebane,994591.743629,770.344157,2000,Maize,...,0.4005,0.6228,13.0,63.0,113.833336,0.773942,17854.916,25.483335,9.0,2.3
5404,11443,38.541884,-16.795183,Mozambique,MOZ,Pebane,994591.743629,770.344157,2000,Maize,...,0.3856,0.5980,14.0,62.0,104.083336,0.519956,17967.250,25.358332,9.0,1.8
5417,11588,39.011112,-16.886089,Mozambique,MOZ,Pebane,994591.743629,770.344157,2000,Maize,...,0.3933,0.6280,10.0,61.0,98.833336,0.459132,18114.750,25.375000,9.0,2.4
5429,11859,38.165334,-17.065165,Mozambique,MOZ,Pebane,994591.743629,770.344157,2000,Maize,...,0.3632,0.5864,8.0,62.0,114.750000,1.108307,17843.166,25.258333,11.0,2.3


#### Set column order

In [25]:
flat_input.columns

Index(['alloc_key', 'lon', 'lat', 'country', 'c_code', 'state', 'statearea_ha',
       'perimeter_km', 'year', 'crop', 'yield', 'harv_area_ha',
       'production_ha', 'BDRICM', 'BLD', 'CLYPPT', 'DRAINFAO', 'elevation',
       'EVI', 'NDVI', 'ORCDRC', 'PHIHOX', 'prec', 'slope', 'srad', 'tavg',
       'TEXMHT', 'wind'],
      dtype='object')

In [26]:
flat_input = change_column_order(flat_input, 'alloc_key', 0)
flat_input = change_column_order(flat_input, 'state', 1)
flat_input = change_column_order(flat_input, 'lon', 2)
flat_input = change_column_order(flat_input, 'lat', 3)
flat_input = change_column_order(flat_input, 'country', 4)
flat_input = change_column_order(flat_input, 'c_code', 5)

flat_input = change_column_order(flat_input, 'statearea_ha', 6)
flat_input = change_column_order(flat_input, 'perimeter_km', 7)
flat_input = change_column_order(flat_input, 'year', 8)
flat_input = change_column_order(flat_input, 'crop', 9)
flat_input = change_column_order(flat_input, 'harv_area_ha', 10)
flat_input = change_column_order(flat_input, 'yield', 11)
flat_input = change_column_order(flat_input, 'production_ha', 12)

flat_input = change_column_order(flat_input, 'tavg', 13)
flat_input = change_column_order(flat_input, 'srad', 14)
flat_input = change_column_order(flat_input, 'prec', 15)
flat_input = change_column_order(flat_input, 'wind', 16)

flat_input = change_column_order(flat_input, 'PHIHOX', 17)
flat_input = change_column_order(flat_input, 'BDRICM', 18)
flat_input = change_column_order(flat_input, 'BLD', 19)
flat_input = change_column_order(flat_input, 'CLYPPT', 20)
flat_input = change_column_order(flat_input, 'TEXMHT', 21)
flat_input = change_column_order(flat_input, 'ORCDRC', 22)
flat_input = change_column_order(flat_input, 'DRAINFAO', 23)

flat_input = change_column_order(flat_input, 'EVI', 24)
flat_input = change_column_order(flat_input, 'NDVI', 25)

flat_input = change_column_order(flat_input, 'slope', 26)
flat_input = change_column_order(flat_input, 'elevation', 27)

In [27]:
flat_input.head()

,alloc_key,state,lon,lat,country,c_code,statearea_ha,perimeter_km,year,crop,...,BDRICM,BLD,CLYPPT,TEXMHT,ORCDRC,DRAINFAO,EVI,NDVI,slope,elevation
76,8082,Ribaue,38.453855,-14.625298,Mozambique,MOZ,644555.756771,406.138864,2000,Maize,...,111.0,1370.0,17.0,9.0,11.0,5.0,0.3034,0.5880,0.785641,423.0
91,8497,Ribaue,37.988363,-14.894948,Mozambique,MOZ,644555.756771,406.138864,2000,Maize,...,138.0,1380.0,16.0,9.0,12.0,5.0,0.3229,0.5771,0.821979,594.0
98,8636,Ribaue,37.894945,-14.984914,Mozambique,MOZ,644555.756771,406.138864,2000,Maize,...,124.0,1380.0,14.0,9.0,10.0,5.0,0.3155,0.5116,1.964763,661.0
100,8638,Ribaue,38.080936,-14.985740,Mozambique,MOZ,644555.756771,406.138864,2000,Maize,...,129.0,1430.0,14.0,9.0,9.0,5.0,0.3180,0.4625,1.473183,671.0
149,9772,Mogovolas,39.384372,-15.710531,Mozambique,MOZ,454898.198313,335.429510,2000,Maize,...,140.0,1470.0,11.0,9.0,8.0,5.0,0.3383,0.5690,1.323050,175.0


In [28]:
flat_input.columns

Index(['alloc_key', 'state', 'lon', 'lat', 'country', 'c_code', 'statearea_ha',
       'perimeter_km', 'year', 'crop', 'harv_area_ha', 'yield',
       'production_ha', 'tavg', 'srad', 'prec', 'wind', 'PHIHOX', 'BDRICM',
       'BLD', 'CLYPPT', 'TEXMHT', 'ORCDRC', 'DRAINFAO', 'EVI', 'NDVI', 'slope',
       'elevation'],
      dtype='object')

#### Final column fixing

In [29]:
# give crop name
crop_modelled ="maize"

# Dropping columns
flat_input = flat_input.drop(["country"], axis=1)
flat_input = flat_input.drop(["c_code"], axis=1)
flat_input = flat_input.drop(["perimeter_km"], axis=1)
flat_input = flat_input.drop(["year"], axis=1)
flat_input = flat_input.drop(["crop"], axis=1)
flat_input = flat_input.drop(["yield"], axis=1)
flat_input = flat_input.drop(["production_ha"], axis=1)


#Remaning columns
flat_input.rename(columns={'state': 'NAME'}, inplace=True)
flat_input.rename(columns={'statearea_ha': 'statearea'}, inplace=True)
flat_input.rename(columns={'harv_area_ha': crop_modelled}, inplace=True)


In [30]:
flat_input

,alloc_key,NAME,lon,lat,statearea,maize,tavg,srad,prec,wind,...,BDRICM,BLD,CLYPPT,TEXMHT,ORCDRC,DRAINFAO,EVI,NDVI,slope,elevation
76,8082,Ribaue,38.453855,-14.625298,644555.756771,11226.0,24.358332,17325.250,87.583336,1.2,...,111.0,1370.0,17.0,9.0,11.0,5.0,0.3034,0.5880,0.785641,423.0
91,8497,Ribaue,37.988363,-14.894948,644555.756771,11226.0,23.341667,16929.334,90.083336,1.2,...,138.0,1380.0,16.0,9.0,12.0,5.0,0.3229,0.5771,0.821979,594.0
98,8636,Ribaue,37.894945,-14.984914,644555.756771,11226.0,22.891668,16879.000,92.083336,1.3,...,124.0,1380.0,14.0,9.0,10.0,5.0,0.3155,0.5116,1.964763,661.0
100,8638,Ribaue,38.080936,-14.985740,644555.756771,11226.0,23.041666,16906.166,93.750000,1.2,...,129.0,1430.0,14.0,9.0,9.0,5.0,0.3180,0.4625,1.473183,671.0
149,9772,Mogovolas,39.384372,-15.710531,454898.198313,3914.0,25.283333,18582.834,88.416664,1.2,...,140.0,1470.0,11.0,9.0,8.0,5.0,0.3383,0.5690,1.323050,175.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5373,11858,Pebane,38.071366,-17.064756,994591.743629,7934.0,25.483335,17854.916,113.833336,2.3,...,135.0,1420.0,16.0,9.0,13.0,5.0,0.4005,0.6228,0.773942,25.0
5404,11443,Pebane,38.541884,-16.795183,994591.743629,7934.0,25.358332,17967.250,104.083336,1.8,...,135.0,1370.0,16.0,9.0,14.0,4.0,0.3856,0.5980,0.519956,44.0
5417,11588,Pebane,39.011112,-16.886089,994591.743629,7934.0,25.375000,18114.750,98.833336,2.4,...,167.0,1440.0,18.0,9.0,10.0,4.0,0.3933,0.6280,0.459132,48.0
5429,11859,Pebane,38.165334,-17.065165,994591.743629,7934.0,25.258333,17843.166,114.750000,2.3,...,148.0,1420.0,10.0,11.0,8.0,5.0,0.3632,0.5864,1.108307,47.0


In [31]:
# This part prints full results

path = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input"
name_of_flat_input_file = "flat_input_Maize_10km"

flat_input.to_csv(os.path.join(path,"{c}.csv".format(c=name_of_flat_input_file)), index=False)